In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [10]:
tf.reset_default_graph()
n_input = 28  # 每一行有28 个像素点
max_time = 28 # 一共有28行像素点
lstm_size = 100  # 一共定义100 个隐藏cell
n_class = 10  # 输出10个分类
batch_size = 50 #定义batch size 为50
n_batch = mnist.train.num_examples // batch_size

################################################
## 定义变量
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
weights = tf.Variable(tf.truncated_normal([lstm_size, n_class], stddev=0.1))
biases = tf.Variable(tf.constant(0.1, shape=[n_class]))
#################################################

################################################
# 定义主要网络结构
def LSTM(X, weights, biases):
    inputs = tf.reshape(X, [-1, max_time, n_input])
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)  # 定义lstm的基本cell
    # final_state[0] 是cell state
    # final_state[1] 是hidden state
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1], weights) + biases)
    return results
################################################

######################################
## 计算
prediction = LSTM(x, weights, biases)
cross_entroy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
###################################################
## 定义优化函数
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entroy) # 使用AdamOptimizer 优化
###################################################
### 计算准确率
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))  # 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
##############################################


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys})
            
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("Iter " + str(epoch) + ", testing accuracy= " + str(acc))

Iter 0, testing accuracy= 0.6757
Iter 1, testing accuracy= 0.8692
Iter 2, testing accuracy= 0.9027
Iter 3, testing accuracy= 0.9132
Iter 4, testing accuracy= 0.923
Iter 5, testing accuracy= 0.9324
Iter 6, testing accuracy= 0.9388
Iter 7, testing accuracy= 0.9402
Iter 8, testing accuracy= 0.9457
Iter 9, testing accuracy= 0.9496
Iter 10, testing accuracy= 0.9536
Iter 11, testing accuracy= 0.9423
Iter 12, testing accuracy= 0.9518
Iter 13, testing accuracy= 0.9579
Iter 14, testing accuracy= 0.9624
Iter 15, testing accuracy= 0.9595
Iter 16, testing accuracy= 0.9615
Iter 17, testing accuracy= 0.9641
Iter 18, testing accuracy= 0.9601
Iter 19, testing accuracy= 0.9606
Iter 20, testing accuracy= 0.9669
